<a href="https://colab.research.google.com/github/jcppaula/Portfolio-Data-Science/blob/main/Business_Intelligence_com_Google_Cloud_Products_com_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Business Intelligence com Python

---

# Introdução

Recém contratado em uma empresa de pequeno porte de Araraquara, com 15 funcionários, você assumiu uma vaga para ajudar a empresa a se modernizar, automatizar processos humanos e gerar inteligência de negócio.

Você está motivado na nova missão, e tem compartilhados *cases* publicados em *blogs* para seus colegas de trabalho, mostrando como usam o Python por aí e como "dados são o novo petróleo".

<img src="https://raw.githubusercontent.com/carlosfab/novo-python-do-zero/main/img/9bdccce68226641d9bda0f9c6aabe96e.jpg">

Nesse seu primeiro dia de trabalho, que coincidiu com a véspera de um feriado prolongado, você recebeu poucas tarefas. Com todos já naquele clima de calmaria que antecede todos os feriados nacionais, você olha a lista e fica aliviado em ver que são coisas relativamente tranquilas.

Mais do que apenas motivado, você está confiante nas suas habilidades. Afinal, não foram poucos os projetos que você fez nessa trilha de aprendizado; bastou uma passada rápida com os olhos para você concluir que o `pandas`, `numpy` e mais alguma biblioteca como `geopandas`, deveriam ser suficientes para resolver esses pepinos rapidamente.

Caneca de café reabastecida, você volta para a sala e faz uma pergunta que seria "básica" para você - mas que no mundo real, foi capaz de abalar toda aquela confiança com os projetos do seu portfólio:

**Onde estão os dados? Em que pasta eu acho as planilhas?**




## Projeto *Business Intelligence com GCP e Python*

A partir de agora nós iremos contextualizar o projeto do módulo. Apesar de ser uma situação fictícia, é próxima à realidade que os novos profissionais enfrentam ao lideram com problemas fora da CNTP (condições normais de temperatura e pressão).

Uma vez que você foi apresentado ao contexto do problema, recebeu os arquivos e entendeu a sua missão geral, irei apresentar as missões específicas do seu *emprego*.

Ao final deste, você será apresentado a algumas tarefas, onde terá que resolver sozinho e sem nenhum gabarito. Quero reforçar a importância dessas tarefas ao final, pois são elas que irão desenvolver a sua capacidade de pesquisa e auto-didatismo.


<center><img src="https://raw.githubusercontent.com/carlosfab/novo-python-do-zero/main/img/28955874330_6cf42a2c54_b.jpg" width= 200></center>


<br><br>

### ***📋 CHECKLIST* DO PROJETO**
✅ Transformar o pdf da planilha escaneada (`enderecos.pdf`) contendo os endereços dos clientes, em um DataFrame.
<br>✅ Obter as coordenadas (Latitude/Longitude) a partir dos endereços dos clientes.
<br>✅ Unificar os endereços com os dados pessoais dos clientes (`clientes_dados.csv`), a partir da coluna `id_cliente`.
<br>✅ Verificar a distância dos clientes até o local do evento, chamado "Estádio da Fonte Luminosa" de Araraquara.

<br>

Assim como foi explicado no *briefing* do módulo, nós iremos lidar com os arquivos **dentro do seu Google Drive pessoal**, em uma pasta qualquer que você tenha escolhido.

Também usaremos a autenticação do Document AI por meio das chaves que foram geradas quando configuramos o projeto e autorizamos as APIs.

Então, certifique-se de ter feito o upload dos seguintes arquivos no Google Drive:
 * `clientes_dados.csv`
 * `enderecos.pdf`
 * `sua-chave-gerada.json` (verifique o nome do seu arquivo gerado)


> Após executar as primeiras célular abaixo, será necessário autorizar o Colab a acessar o volume do Google Drive. Depois, é só navegar até a pasta do projeto, contendo os arquivos supracitados.

### Instalação das bibliotecas e autenticação

Com base no conhecimento adquirido até então, e considerando que você configurou corretamente o ambiente do *Console* e ativou as APIs necessárias, vamos começar instalando as bibliotecas que serão utilizadas e analisar os arquivos que teremos que lidar.

Lembre que, assim como comentamos na aula de Document AI, após instalar a biblioteca `google-cloud-documentai` você precisará reiniciar o *kernel* do Colab.

In [ ]:
# instalar produtos do Google Cloud
!pip install --upgrade google-cloud-documentai
!pip install googlemaps
!pip install --upgrade geopandas -q
!pip install mapclassify -q
!pip install folium -q

In [ ]:
# Importar as bibliotecas necessárias
from google.oauth2 import service_account
from google.colab import drive
from typing import List, Sequence
import pandas as pd
from google.api_core.client_options import ClientOptions
from google.cloud import documentai_v1 as documentai
import json
import googlemaps
from datetime import datetime, date
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from folium.plugins.heat_map import HeatMap
from shapely.geometry import Point, LineString, Polygon


O que a linha `%env GOOGLE_APPLICATION_CREDENTIALS=$key_path` fará é criar uma variável no ambiente do servidor, com referência ao arquivo *json* que contém os dados para o Colab autenticar junto aos produtos do seu projeto no Google Cloud.

Obviamente estamos em um ambiente didático, mas fica o aviso: nunca compartilhe esse arquivo. Da mesma maneira, quando estiver lidando com *tokens* diretamente no *notebook*, tome cuidado para não publicar ou subir o arquivo em um repositório público.

Qualquer pessoa que detenha essa informação, irá ter acesso aos produtos ativados - ou seja, a fatura poderá vir alta!

In [ ]:
# Mount Google Drive
drive.mount('/content/drive/')

# Caminho até a sua pasta de projetos
PROJECT_PATH = '/content/drive/MyDrive/Sigmoidal/Python do Zero/Projetos/Modulo 4 - BI'

# Indicar o caminho até o arquivo JSON para autenticação
key_path = '/content/drive/MyDrive/Sigmoidal/Python do Zero/Projetos/Modulo 4 - BI/minhachave.json'

# Inserir uma variável no ambiente
%env GOOGLE_APPLICATION_CREDENTIALS=$key_path

Mounted at /content/drive/
env: GOOGLE_APPLICATION_CREDENTIALS=/content/drive/MyDrive/Sigmoidal/Python do Zero/Projetos/Modulo 4 - BI/modulo-4-busines-intelligence-9d9c8283d37c.json


## *Data Ingestion* e *Analytics*

Para atender todas as atribuições do *checklist*, nós iremos cumprir uma sequência de passos intermediários:

1. Converter o arquivo pdf contendo os endereços dos clientes (tabela do Excel impressa e escaneada) para uma estrutura de DataFrame, utilizando o Document AI (Form Parser).
2. Ler o arquivo csv contendo os dados pessoais dos clientes para uma estrutura de DataFrame, utilizando o Pandas.
3. Verificar a integridade dos dados e se os tipos são adequados a cada coluna.
4. Combinar os dois DataFrames em uma única variável, usando a coluna `id_cliente` como referência.
5. Obter as coordenadas geográficas de cada entrada do *dataset* unificado, por meio do Google Maps.
6. Gerar um novo arquivo `dados_completos.csv` e deixá-lo salvo no diretório do projeto, no Google Drive.
7. Obter a distância e tempo de deslocamento de cada cliente até o ponto marcante "Estádio da Fonte Luminosa, Araraquara - SP".
8. Você irá prosseguir adiante, aprofundando nas análises e gerando mais *insights* para a empresa.


### 1. Converter *pdf* em *DataFrame*

Para isso será usado a API do Google, *Document AI*, com ela podemos transformar dados não estruturados, no nosso caso uma planilha escaneada em formato PDF em um DataFrame.

In [ ]:
"""
Fazer uma requisição para processamento ao Document AI
"""


project_id = '549126243421' 
location = 'us' 
processor_id = '9853549863e7f0a1' 
file_path = '/content/drive/MyDrive/Sigmoidal/Python do Zero/Projetos/Modulo 4 - BI/enderecos.pdf'
mime_type = 'application/pdf' 


def online_process(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
) -> documentai.Document:
    """
    Processes a document using the Document AI Online Processing API.
    """

    # Instantiates a client
    docai_client = documentai.DocumentProcessorServiceClient(
        client_options=ClientOptions(
            api_endpoint=f"{location}-documentai.googleapis.com"
        )
    )

    # The full resource name of the processor, e.g.:
    # projects/project-id/locations/location/processor/processor-id
    # You must create new processors in the Cloud Console first
    resource_name = docai_client.processor_path(project_id, location, processor_id)

    # Read the file into memory
    with open(file_path, "rb") as file:
        file_content = file.read()

    # Load Binary Data into Document AI RawDocument Object
    raw_document = documentai.RawDocument(content=file_content, mime_type=mime_type)

    # Configure the process request
    request = documentai.ProcessRequest(name=resource_name, raw_document=raw_document)

    # Use the Document AI client to process the sample form
    result = docai_client.process_document(request=request)

    return result.document


def get_table_data(
    rows: Sequence[documentai.Document.Page.Table.TableRow], text: str
) -> List[List[str]]:
    """
    Get Text data from table rows
    """
    all_values: List[List[str]] = []
    for row in rows:
        current_row_values: List[str] = []
        for cell in row.cells:
            current_row_values.append(
                text_anchor_to_text(cell.layout.text_anchor, text)
            )
        all_values.append(current_row_values)
    return all_values


def text_anchor_to_text(text_anchor: documentai.Document.TextAnchor, text: str) -> str:
    """
    Document AI identifies table data by their offsets in the entirity of the
    document's text. This function converts offsets to a string.
    """
    response = ""
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    for segment in text_anchor.text_segments:
        start_index = int(segment.start_index)
        end_index = int(segment.end_index)
        response += text[start_index:end_index]
    return response.strip().replace("\n", " ")


# Fazer a requisição e enviar o pdf ao processador Form Parser
document = online_process(
    project_id,
    location,
    processor_id,
    file_path,
    mime_type
)

# Extrair cabeçalho e linhas da tabela
header_row_values: List[List[str]] = []
body_row_values: List[List[str]] = []

# caso houvesse mais páginas no documento pdf
for page in document.pages:
    for table in page.tables:
        header_row_values = get_table_data(table.header_rows, document.text)
        body_row_values = get_table_data(table.body_rows, document.text)

# criar um Pandas Dataframe para armazenar os valores em formato tabular
df_enderecos = pd.DataFrame(
    data=body_row_values,
    columns=header_row_values[0],
)

In [ ]:
df_enderecos.head()

,id_cliente,logradouro,numero,bairro,cep,cidade,uf
0,1,Rua Bento de Miranda Mello,356,Residencial Olivio Benassi,15993-037,Matão,SP
1,2,Rua Ricardo Arruda Ribeiro,745,Água Branca,13425-100,Piracicaba,SP
2,3,Rua Dona Sylvia Finco Costa,870,Jardim Tapajós,12945-080,Atibaia,SP
3,4,Rua Axicará,433,Vila Isolina Mazzei,02081-080,São Paulo,SP
4,5,Rua Diamantino Armando,245,Vila Esperança,18703-800,Avaré,SP


### 2. Carregar *csv* em *DataFrame*

Vamos carregar a plhanilha em formato *cvs*, com os dados dos clientes que após alguns ajustes será unificada com a planilha endereços obtida a partir do *PDF*. 

In [ ]:
df_dados = pd.read_csv('/content/drive/MyDrive/Sigmoidal/Python do Zero/Projetos/Modulo 4 - BI/clientes_dados.csv', sep=";")

In [ ]:
df_dados.head()

,id_cliente,nome,email,sexo,cpf,data_nasc
0,1,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,06/01/1984
1,2,Andrea Débora Campos,andrea-campos75@caporal.biz,Feminino,782.328.538-08,06/01/1943
2,3,Rosa Jéssica Isis Barros,rosa.jessica.barros@optovac.com.br,Feminino,129.102.618-59,04/02/1970
3,4,Vinicius Carlos Eduardo da Rocha,viniciuscarlosdarocha@carreira.com.br,Masculino,437.270.578-66,22/06/1969
4,5,Cecília Milena Elisa Vieira,cecilia-vieira98@amure.com.br,Feminino,726.017.228-27,07/09/1948


### 3. Verificar e transformar dados

In [ ]:
# verificar os formatos das colunas do DataFrame df_enderecos
df_enderecos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  30 non-null     object
 1   logradouro  30 non-null     object
 2   numero      30 non-null     object
 3   bairro      30 non-null     object
 4   cep         30 non-null     object
 5   cidade      30 non-null     object
 6   uf          30 non-null     object
dtypes: object(7)
memory usage: 1.8+ KB


In [ ]:
# verificar os formatos das colunas do DataFrame df_dados
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  30 non-null     int64 
 1   nome        30 non-null     object
 2   email       30 non-null     object
 3   sexo        30 non-null     object
 4   cpf         30 non-null     object
 5   data_nasc   30 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.5+ KB


In [ ]:
# mudar o formato da coluna id_cliente para 'int64'
df_enderecos['id_cliente'] = df_enderecos['id_cliente'].astype('int64')

In [ ]:
# mudar o formato da coluna id_cliente para 'datetime'
df_dados['data_nasc'] = pd.to_datetime(df_dados['data_nasc'], format='%d/%m/%Y')

In [ ]:
# verificar os formatos das colunas do DataFrame df_endereços
df_enderecos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_cliente  30 non-null     int64 
 1   logradouro  30 non-null     object
 2   numero      30 non-null     object
 3   bairro      30 non-null     object
 4   cep         30 non-null     object
 5   cidade      30 non-null     object
 6   uf          30 non-null     object
dtypes: int64(1), object(6)
memory usage: 1.8+ KB


In [ ]:
# verificar os formatos das colunas do DataFrame df_dados
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_cliente  30 non-null     int64         
 1   nome        30 non-null     object        
 2   email       30 non-null     object        
 3   sexo        30 non-null     object        
 4   cpf         30 non-null     object        
 5   data_nasc   30 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.5+ KB


### 4. Combinar em um *DataFrame*

Enfim agora iremos unificar as duas planilhas, combinando os dois DataFrames com base no ídice, que será definido pela coluna `id_cliente`.

In [ ]:
df_completo = pd.merge(df_dados, df_enderecos, on='id_cliente')

In [ ]:
df_completo.head()

,id_cliente,nome,email,sexo,cpf,data_nasc,logradouro,numero,bairro,cep,cidade,uf
0,1,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,1984-01-06,Rua Bento de Miranda Mello,356,Residencial Olivio Benassi,15993-037,Matão,SP
1,2,Andrea Débora Campos,andrea-campos75@caporal.biz,Feminino,782.328.538-08,1943-01-06,Rua Ricardo Arruda Ribeiro,745,Água Branca,13425-100,Piracicaba,SP
2,3,Rosa Jéssica Isis Barros,rosa.jessica.barros@optovac.com.br,Feminino,129.102.618-59,1970-02-04,Rua Dona Sylvia Finco Costa,870,Jardim Tapajós,12945-080,Atibaia,SP
3,4,Vinicius Carlos Eduardo da Rocha,viniciuscarlosdarocha@carreira.com.br,Masculino,437.270.578-66,1969-06-22,Rua Axicará,433,Vila Isolina Mazzei,02081-080,São Paulo,SP
4,5,Cecília Milena Elisa Vieira,cecilia-vieira98@amure.com.br,Feminino,726.017.228-27,1948-09-07,Rua Diamantino Armando,245,Vila Esperança,18703-800,Avaré,SP


In [ ]:
df_completo.set_index('id_cliente', inplace=True)

In [ ]:
df_completo.head()

,nome,email,sexo,cpf,data_nasc,logradouro,numero,bairro,cep,cidade,uf
id_cliente,,,,,,,,,,,
1,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,1984-01-06,Rua Bento de Miranda Mello,356,Residencial Olivio Benassi,15993-037,Matão,SP
2,Andrea Débora Campos,andrea-campos75@caporal.biz,Feminino,782.328.538-08,1943-01-06,Rua Ricardo Arruda Ribeiro,745,Água Branca,13425-100,Piracicaba,SP
3,Rosa Jéssica Isis Barros,rosa.jessica.barros@optovac.com.br,Feminino,129.102.618-59,1970-02-04,Rua Dona Sylvia Finco Costa,870,Jardim Tapajós,12945-080,Atibaia,SP
4,Vinicius Carlos Eduardo da Rocha,viniciuscarlosdarocha@carreira.com.br,Masculino,437.270.578-66,1969-06-22,Rua Axicará,433,Vila Isolina Mazzei,02081-080,São Paulo,SP
5,Cecília Milena Elisa Vieira,cecilia-vieira98@amure.com.br,Feminino,726.017.228-27,1948-09-07,Rua Diamantino Armando,245,Vila Esperança,18703-800,Avaré,SP


### 5. Obter coordenadas geográficas

Para obter as coordenadas a partir dos endereços do nosso DataFrame, será usado a API Google Maps Plataform.

In [ ]:
# API do Google Maps
gmaps = googlemaps.Client(key='minha chave')

In [ ]:
# Gerar coluna com endereços completos, para minimizar conflitos
enderecos_completos = df_completo["logradouro"] + " , " + df_completo["numero"] + " - " + df_completo["bairro"] + " - " + df_completo["cidade"] + " , " + df_completo["uf"]

In [ ]:
# exemplo de endereço completo
enderecos_completos.iloc[0]

'Rua Bento de Miranda Mello , 356 - Residencial Olivio Benassi - Matão , SP'

In [ ]:
# Obter coordenadas de maneira iterativa
coordenadas = []

# percorrer entradas individuais e fazer requisições individuais
for i, j in enumerate(enderecos_completos):
    geocode_obj = gmaps.geocode(j)

    # extrair apenas lat/lon
    lat = geocode_obj[0]['geometry']['location']['lat']
    lon = geocode_obj[0]['geometry']['location']['lng']

    # adicionar lat/lon à lista
    coordenadas.append([lat, lon])

In [ ]:
geocode_obj[0]['geometry']['location']

{'lat': -23.6826179, 'lng': -46.3587757}

In [ ]:
# converter para DataFrame
df_coordenadas = pd.DataFrame(coordenadas, columns=['lat', 'lon'])

# Obter mesmo índice que o dataframe completo
df_coordenadas.set_index(df_coordenadas.index + 1, inplace=True)

### 6. Exportar *csv* com dados completos

In [ ]:
# combinar coordenadas aos dados existentes, em relação ao axis=1
df_completo = pd.concat([df_completo, df_coordenadas], axis=1)

In [ ]:
# exportar para arquivo csv dentro do Google Drive
df_completo.to_csv('/content/drive/MyDrive/Sigmoidal/Python do Zero/Projetos/Modulo 4 - BI/dados_completos.csv')

### 7. Calcular distância até o evento

Mais ums vez será usado a API do Google Maps, agora para calcular a distancia a partir das informações de latitude e longitude obtidas anteriormente, até o Estádio da Fonte Luminosa, Araraquara - SP. 



In [ ]:
local_do_evento = "Estádio da Fonte Luminosa, Araraquara - SP"

In [ ]:
# obter informações sobre o local do evento
gmaps.geocode(local_do_evento)

In [ ]:
# exemplo de como calcular distancia/tempo entre pontos, com data específica
dia_do_evento = datetime(2022, 11, 11)
directions_result = gmaps.directions("-21.620162, -48.366012",
                                     local_do_evento,
                                     mode="driving",
                                     departure_time=dia_do_evento)

In [ ]:
int(directions_result[0]['legs'][0]['distance']['value'] / 1000)

32

In [ ]:
directions_result[0]['legs'][0]['duration']['value'] / 60

35.88333333333333

In [ ]:
# obter todas as distâncias e tempos de estrada
lista_distancias = []
dia_do_evento = datetime(2022, 11, 11)

# percorrer entradas individuais e fazer requisições individuais
for i, j in enumerate(enderecos_completos):
    directions_result = gmaps.directions(j,
                                     local_do_evento,
                                     mode="driving",
                                     departure_time=dia_do_evento)

    # adicionar às listas
    distancia = directions_result[0]['legs'][0]['distance']['value']
    tempo = directions_result[0]['legs'][0]['duration']['value'] / 60
    lista_distancias.append([distancia, tempo])

In [ ]:
# converter para DataFrame
df_distancias = pd.DataFrame(lista_distancias, columns=['distancia', 'tempo'])

# Obter mesmo índice que o dataframe completo
df_distancias.set_index(df_distancias.index + 1, inplace=True)

In [ ]:
# combinar coordenadas aos dados existentes, em relação ao axis=1
df_evento = pd.concat([df_completo['nome'], df_distancias], axis=1)

In [ ]:
df_evento.head()

,nome,distancia,tempo
1,Sebastiana Tânia Mendes,32497,35.883333
2,Andrea Débora Campos,144817,115.666667
3,Rosa Jéssica Isis Barros,253539,186.050000
4,Vinicius Carlos Eduardo da Rocha,278701,207.883333
5,Cecília Milena Elisa Vieira,185085,156.533333


In [ ]:
# converter metros em kilometros (inteiro)
df_evento['distancia'] = (df_evento['distancia'] / 1000).astype('int64')

# converter minutos em horas
df_evento['tempo'] = (df_evento['tempo'] / 60)

In [ ]:
df_evento.head()

,nome,distancia,tempo
1,Sebastiana Tânia Mendes,32,0.598056
2,Andrea Débora Campos,144,1.927778
3,Rosa Jéssica Isis Barros,253,3.100833
4,Vinicius Carlos Eduardo da Rocha,278,3.464722
5,Cecília Milena Elisa Vieira,185,2.608889


### 8. Verificar o convidado que mora mais longe do evento


In [ ]:
df_evento.groupby(['nome']).distancia.mean().sort_values(ascending=False)[:5]

nome
Renato Vicente Antonio Freitas     370.0
Noah Murilo Victor Ferreira        339.0
Eliane Simone Isabelle Aparício    318.0
Oliver Noah Fernando Bernardes     317.0
Samuel Anthony Costa               302.0
Name: distancia, dtype: float64

Calculada a distância de cada cliente até o local do evento, foi organizado por ordem crescente, e podemos verificar que o senhor Renato Vicente Antonio Freitas, é o cliente que mora mais longe, totalizando uma distância de 370 quilometros.

In [ ]:
df_evento['distancia'].describe()

count     30.000000
mean     241.600000
std       80.677134
min       32.000000
25%      187.750000
50%      265.000000
75%      295.750000
max      370.000000
Name: distancia, dtype: float64

Com a função describe, podemos ter uma ideia melhor das distâncias que os cliente estão do evendo, como por exemplo a média das distâncias.

###9.  Calcular a idade média dos seus clientes

In [ ]:
# separar um dataframe para trabalhar a idade
df_idade = df_completo[['nome', 'data_nasc']]

In [ ]:
# obter idades com base na data de nascimento

lista_idades = []
data_nascimento = df_idade['data_nasc']
data_hoje = datetime.today()

# percorrer entradas individuais e fazer requisições individuais
for i, j in enumerate(df_idade):
        idade = (data_hoje - data_nascimento).astype('timedelta64[Y]')

# adicionar às listas
lista_idades.append([idade])

In [ ]:
# combinar as idades aos dados existentes, em relação ao axis=1
df_idade['idade'] = idade
df_idade['idade'] = (df_idade['idade']).astype('int')

In [ ]:
df_idade.head()

,nome,data_nasc,idade
1,Sebastiana Tânia Mendes,1984-01-06,38
2,Andrea Débora Campos,1943-01-06,79
3,Rosa Jéssica Isis Barros,1970-02-04,52
4,Vinicius Carlos Eduardo da Rocha,1969-06-22,53
5,Cecília Milena Elisa Vieira,1948-09-07,74


In [ ]:
df_idade.groupby(['nome']).idade.mean().sort_values(ascending=False)[:5]

nome
Augusto Vicente Breno Moreira     80.0
Andrea Débora Campos              79.0
Renato Vicente Antonio Freitas    78.0
Emilly Marlene Gabrielly Silva    76.0
Caleb Calebe Pires                75.0
Name: idade, dtype: float64

Calculada as idades dos clientes e organizadas por ordem crescente, podemos verificar que o senhor Augusto Vicente Breno Moreira, com 80 anos é nosso cliente mais velho.

In [ ]:
# calcular a média de idade dos convidados
media_idades = int(df_idade['idade'].mean())
print('Média de Idades:', media_idades)

Média de Idades: 55


Com a função .mean(), verificamos que a média de idades dos cliente é de 55 anos.

###10.  Plotar as informações geográficas (Lat/Lon) em um mapa usando a Biblioteca Folium

In [ ]:
#verificar o Data Frame completo com as coordenadas
df_completo.head(1)

,nome,email,sexo,cpf,data_nasc,logradouro,numero,bairro,cep,cidade,uf,lat,lon
1,Sebastiana Tânia Mendes,sebastiana_tania_mendes@trbvm.com,Feminino,938.319.088-42,1984-01-06,Rua Bento de Miranda Mello,356,Residencial Olivio Benassi,15993-037,Matão,SP,-21.620162,-48.366012


In [ ]:
# criar um um Data Frame com a localização Arena Fonte Luminosa Araraquara 
data = {'lat': [-21.7752895],'lon':[-48.1724652]}
df_araraquara = pd.DataFrame(data, index=['Arena Fonte Luminosa Araraquara'])
print(df_araraquara)

                                       lat        lon
Arena Fonte Luminosa Araraquara -21.775289 -48.172465


In [ ]:
# criando as variáveis necessárias com base no Data Frame df_completo
latitude = df_completo.lat.values
longitude = df_completo.lon.values
listings = df_completo.nome.values

In [ ]:
# criar o mapa usando a biblioteca Folium
sp = folium.Map(location=[-22.5532495,-50.8800911],
                zoom_start=7, tiles='cartodbpositron')

# criar os marcadores com as coordenadas dos endereços dos clientes
for lat, lon, listing in zip(latitude, longitude, listings):
    folium.Marker(location=[float(lat), float(lon)], popup=listing).add_to(sp)

In [ ]:
# criando as variáveis necessárias com base no Data Frame df_araraquara
latitude = df_araraquara.lat.values
longitude = df_araraquara.lon.values
listings = df_araraquara.index.values

In [ ]:
# criar o marcador com as coordenadas da Arena Fonte Luminosa Araraquara
for lat, lon, listing in zip(latitude, longitude, listings):
    folium.Marker(location=[float(lat), float(lon)], popup=listing, icon=folium.Icon(color="red")).add_to(sp)

sp

###11. Plotar um HeatMap usando a Biblioteca Folium

In [ ]:
# plotar um HeatMap

fmap = folium.Map(location=[-22.5532495,-50.8800911], zoom_start=7, tiles='cartodbpositron')

heat_map = HeatMap(df_completo[['lat', 'lon']])

fmap.add_child(heat_map)

fmap